In [1]:
import pandas as pd
import numpy as np
import os

os.chdir('/Users/gianni/projects/mnk-cleaning-analysis/')

os.chdir('/Users/gianni/Google Drive/Bas Zahy Gianni - Games/Data')

In [16]:
oc = [
    'index', 'subject', 'color', 'gi', 'mi', 
    'status', 'bp', 'wp', 'response', 'rt',
    'time', 'mouse_t', 'mouse_x'
]

fc = [
    'subject', 'is_comp', 'color', 'status',
    'bp', 'wp', 'response', 'rt', 'gi', 'mi',
    'computer', 'human', 'time'
]

mc = ['subject', 'color', 'bp', 'wp', 'response', 'rt', 'condition']

class Data():
    """ Data is the primary object for holding experimental data. It also contains functions
        for the loading, cleaning, augmentation, and export of the data tables. """

    def __init__(self, folder):
        self.data = self.load(folder)

    def load_file(self, folder, file_name, mouse=False):
        """ Initial preparation of data for individual files """
        print(file_name[:-4])

        # load file, drop nuissance columns, remove non-observations
        drop_cols = ['index'] if mouse else ['index', 'mouse_t', 'mouse_x']
        data = pd.read_csv(folder + file_name, names=oc).drop(drop_cols, axis=1)
        drop_status = (data.status != 'dummy') &  (data.status != 'ready') & (data.status != 'draw offer')
        data = data.loc[drop_status, :].copy().reset_index(drop=True)

        # assign unique subject label (from filename) and create separate cols for humans and computers
        sub_filter = data.rt > 0
        comp_filter = data.rt == 0
        first_move_filter = data.bp.map(lambda x: np.array(list(x)).astype(int).sum()==1) #(data.mi == 0) & (data.gi%2 == 0)
        second_move_filter = data.bp.map(lambda x: np.array(list(x)).astype(int).sum()==2) #(data.mi == 1) & (data.gi%2 == 0)
        condition_filter = (data.rt>0)&(data.status == 'playing')
        data.loc[condition_filter, 'condition'] = data.loc[condition_filter, 'subject'].map(lambda x: x[-1])
        data.loc[:, 'condition'] = data.loc[:, 'condition'].fillna(method='ffill')
        
        data.loc[data.rt > 0, 'subject'] = file_name[:-4]
        data.loc[:, 'human'] = file_name[:-4]
        data.loc[:, 'computer'] = np.nan
        data.loc[comp_filter, 'computer'] = data.loc[comp_filter, 'subject']
        data.loc[first_move_filter, 'computer'] = data.loc[second_move_filter, 'computer']
        data.loc[:, 'computer'] = data.loc[:, 'computer'].fillna(method='ffill')
        data.loc[0, 'computer'] = data.loc[1, 'computer']

        return data

    def load(self, folder):
        """ Calls other functions to corrale data and some support information """
        self.exp_name = folder
        files = os.listdir(folder + '/Raw/')
        files = [f for f in files if f[-3:] == 'csv']
#         files =[f for f in files if f[:-4] != 'HH']
        self.subjects = [f[:-4] for f in files]
        self.subject_dict = dict(zip(self.subjects, np.arange(len(self.subjects))))
        data = pd.concat([self.load_file(folder + '/Raw/', f) for f in files])
        data = data.reset_index(drop=True)
        data = self.clean(data)

        return data

    def clean(self, df):
        """ Performs further cleaning that can be done on all data collectively """

        # anonymize subjects
        sub_filter = df.rt > 0 # filter computers out
        df.loc[sub_filter, 'subject'] = df.loc[sub_filter, 'subject'].map(self.subject_dict)
        df.loc[:, 'human'] = df.loc[:, 'human'].map(self.subject_dict)

        # give computers identifiable names
        comp_filter = df.rt == 0
        df.loc[comp_filter, 'subject'] = df.loc[comp_filter, 'subject'].astype(int) + 1000
        df.loc[pd.notnull(df.computer), 'computer'] = df.loc[pd.notnull(df.computer), 'computer'].astype(int) + 1000

        # force remove response from board
        for i in df.loc[df.status != 'EVAL', :].index.values:
            if df.loc[i,"color"] == 0:
                l = list(df.loc[i,"bp"])
                l[df.loc[i, "response"]] = '0'
                df.loc[i,"bp"] = ''.join(l)
            else:
                l = list(df.loc[i,"wp"])
                l[df.loc[i,"response"]] = '0'
                df.loc[i,"wp"] = ''.join(l)

        # force correct colors
        count_pieces = lambda x: np.array([np.array(list(df.loc[i, x])).astype(int).sum() for i in df.index.values])
        df.loc[:, 'color'] = count_pieces('bp') - count_pieces('wp')
        df.loc[:, 'color'] = df.loc[:, 'color'].astype(int).astype(str)

        # add is_comp
        is_computer = lambda x: "0" if x > 0 else "1"
        df.loc[:, 'is_comp'] = df.loc[:, 'rt'].map(is_computer)

        # correct move index in games
        df.loc[df.status.isin(['playing', 'win', 'draw', 'timeout']), 'mi'] = df.loc[df.status.isin(['playing', 'win', 'draw']), 'mi'] - 1
        return df

    def export_individuals(self, folder):
        for s, i in self.subject_dict.items():
            c = self.data.human == i
            d = self.data.loc[c, :].reset_index(drop=True)
            d = d.reindex_axis(self.full_output_columns, axis=1)
            d.to_csv(folder + '/Clean/' + s + '.csv', index=False)

        return None

    def export(self, folder):
        f = folder + 'Clean/_summaries/'
        E = self.data.loc[self.data.status.isin(['playing', 'win', 'draw', 'timeout']), :]
        E.loc[:, fc].to_csv(f + 'all_fields.csv', index=False)
        E.loc[:, mc].to_csv(f + 'model_fields.csv', index=False)
        
        return None

In [17]:
D = Data('./5_tai')

AB
AL
AM
AP
BL
CH
CH2
CR
CS
CW
DT
DW
EW
EY
HH
IRL
JO
JO2
JS
KSV
ML
MP
NP
RN
SC
SS
TB
YE
ZH
ZL


In [19]:
D.export('./5_tai/')

In [21]:
np.array(list("000101")).astype(int).sum()

2

In [6]:
pd.read_csv('./5_tai/Raw/HH.csv')

,1,1474477188600_1,0,0.1,0.2,ready,0.3,0.4,99,0.5,1.47E+12,Unnamed: 11,Unnamed: 12
0,2,77,1,0,0,ready,0.000000e+00,0.000000e+00,99,0,1.470000e+12,NaN,NaN
1,3,1474477188600_1,0,0,1,playing,1.000000e+08,0.000000e+00,27,2059,1.470000e+12,"1474477232929,1474477232936,1474477232944,1474...","421,501;407,501;388,501;371,501;360,501;352,50..."
2,4,77,1,0,2,playing,1.000000e+08,1.000000e+13,22,0,0.000000e+00,NaN,NaN
3,5,1474477188600_1,0,0,3,playing,1.000000e+17,1.000000e+13,18,1827,1.470000e+12,"1474477234858,1474477235128,1474477235135,1474...","327,532;327,531;333,527;342,523;349,520;358,51..."
4,6,77,1,0,4,playing,1.000000e+17,1.000000e+26,9,0,0.000000e+00,NaN,NaN
5,7,1474477188600_1,0,0,5,playing,1.000000e+17,1.000000e+26,28,1069,1.470000e+12,"1474477242368,1474477242528,1474477242536,1474...","305,455;307,455;313,457;322,461;337,469;355,48..."
6,8,77,1,0,6,playing,1.000000e+17,1.000000e+26,21,0,0.000000e+00,NaN,NaN
7,9,1474477188600_1,0,0,7,playing,1.100000e+17,1.000000e+26,19,1445,1.470000e+12,"1474477246658,1474477246785,1474477246792,1474...","408,553;408,560;408,566;408,576;411,588;512,58..."
8,10,77,1,0,8,playing,1.100000e+17,1.100000e+26,10,0,0.000000e+00,NaN,NaN
9,11,1474477188600_1,0,0,9,playing,1.100000e+17,1.100000e+26,29,1615,1.470000e+12,"1474477253381,1474477253512,1474477253520,1474...","395,447;396,453;402,468;413,483;426,503;438,52..."
